In [ ]:
import mediapipe as mp
import numpy as np
import cv2

In [ ]:
mp_drawing  = mp.solutions.drawing_utils

mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection()

mp_face_mesh = mp.solutions.face_mesh
face_mesh    = mp_face_mesh.FaceMesh()

mp_holistic = mp.solutions.holistic
holistic    = mp_holistic.Holistic()

In [ ]:
help(holistic)

In [ ]:
# Holistic
def draw_facemarks(image, results):
    mp_drawing.draw_landmarks(
        image         = image,
        landmark_list = results.face_landmarks,
        #connections   = mp_holistic.FACEMESH_CONTOURS,
        
        landmark_drawing_spec = mp_drawing.DrawingSpec(
                                    color=(0,2,255),
                                    thickness=1,
                                    circle_radius=2),
        
        connection_drawing_spec = mp_drawing.DrawingSpec(
                                color=(128,128,128),
                                thickness=1,
                                circle_radius=1)
        )

# Holistic
def draw_posemarks(image, results):
    mp_drawing.draw_landmarks(
        image = image,
        landmark_list = results.pose_landmarks,
        connections   = mp_holistic.POSE_CONNECTIONS,
        
        landmark_drawing_spec = mp_drawing.DrawingSpec(
            color=(0,230,255),
            thickness=2,
            circle_radius=1),
        
        connection_drawing_spec = mp_drawing.DrawingSpec(
            color=(255,255,255),
            thickness=1,
            circle_radius=1)
        )

# FaceMesh
def draw_face_landmarks(image, results):
    if not results.multi_face_landmarks:
        return
    for face_landmark in results.multi_face_landmarks:
        mp_drawing.draw_landmarks(
            image = image,
            landmark_list = face_landmark,
            connections   = mp_face_mesh.FACEMESH_TESSELATION,

            landmark_drawing_spec = mp_drawing.DrawingSpec(
                color=(255,0,0),
                thickness=2,
                circle_radius=1),
            
            connection_drawing_spec = mp_drawing.DrawingSpec(
                color=(255,255,255),
                thickness=1,
                circle_radius=1)
        )


def draw_face_detection(image, results):
    if not results.detections:
        return
    for landmark in results.detections:
        mp_drawing.draw_landmarks( image,landmark)

#### Model select

In [ ]:
# change here | ONLY FOR LIVE VIDEO

MODEL   = holistic
DRAW_FN = draw_posemarks

### Test on live video

In [ ]:
cam = cv2.VideoCapture(r".\train_videos\train1-s.mp4")
cam.set(cv2.CAP_PROP_POS_FRAMES, (58*30)-1)

while True:
    _, frame = cam.read()
    if not _: break

    results = MODEL.process(frame)    # MODEL
    DRAW_FN(frame, results)

    '''
    ### DRAW
    if results.detections:
        print("DETECTED")
        for landmark in results.detections:
            mp_drawing.draw_landmarks(frame, landmark)
    ###
    '''
    
    cv2.imshow("cam", cv2.resize(frame, (0,0), fx=0.4, fy=0.4))
    key = cv2.waitKey(1)
    if key == ord('q'):
        break

cam.release()
cv2.destroyAllWindows()

In [ ]:
cam.release()
cv2.destroyAllWindows()

In [ ]:
print(results.detections[0])

### Test on video

In [ ]:
video = cv2.VideoCapture(r'.\previous-MLcode\Train_video.mp4')
DIMS  = np.array([500, 880], dtype=int)      #Video crop dimension to subject
SCALE = 0.75

frame_no = 0

while(1):
    #reading video data
    success, frame = video.read()
    if not success: break
    #if frame_no == 120: break     #Testing

    frame_no += 1
    #print(frame_no)
    
    #resizing and cropping
    frame = frame[300:300+DIMS[1], 150:150+DIMS[0]]
    frame = cv2.resize(frame, (0, 0), fx = SCALE, fy = SCALE)

    results = holistic.process(frame)
    draw_facemarks(frame, results)

    results = face_mesh.process(frame)
    draw_face_landmarks(frame, results)

    if not bool(results.face_landmarks):
        print(frame_no)
        

    cv2.imshow("video", frame)
    key = cv2.waitKey(1)
    if key == ord('q'):
        break

video.release()
cv2.destroyAllWindows()

### Test on image

In [ ]:
def draw_holistics(frame, results):
    mp_drawing.draw_landmarks( frame,
        results.face_landmarks,
        mp_holistic.FACEMESH_CONTOURS,
        mp_drawing.DrawingSpec(
            color=(255,0,255),
            thickness=1,
            circle_radius=1),
        
        mp_drawing.DrawingSpec(
            color=(255,255,255),
            thickness=1,
            circle_radius=1)
        )
    
    mp_drawing.draw_landmarks(
        frame,
        results.pose_landmarks,
        mp_holistic.POSE_CONNECTIONS,
        mp_drawing.DrawingSpec(
            color=(0,230,255),
            thickness=2,
            circle_radius=1),
        
        mp_drawing.DrawingSpec(
            color=(255,255,255),
            thickness=1,
            circle_radius=1)
        )

In [ ]:
image = cv2.imread(r".\sample-img\sample1.png")


results = holistic.process(image)
draw_holistics(image, results)

cv2.imshow("here", image)
key = cv2.waitKey(0)
cv2.destroyAllWindows()

# HELPS:

In [ ]:
dir(mp.solutions)

In [ ]:
help(mp_drawing.draw_landmarks)

In [ ]:
dir(results.multi_face_landmarks[0])